In [ ]:
import os
import json
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Set constants

PATH = "data/"
DATA_PATH="crawler_data/data/"
DOMAINS = {
    "nytimes.com": 0,
    "washingtonpost.com": 0,
    "huffpost.com": 0,
    "vox.com": 0,
    "foxnews.com": 1,
    "washingtontimes.com": 1,
    "breitbart.com": 1,
    "nypost.com": 1
}
NUM_ARTICLES_PER_DOMAIN = 500

In [ ]:
def get_label(source_domain):
    return DOMAINS[source_domain] if source_domain in DOMAINS else None

In [ ]:
articles = []

def walk_through_dir(dir):
    for root, dirs, files in os.walk(dir):
        files = (file_path for file_path in files if file_path.endswith(".json"))
        for file_path in files:
            with open(root + "/" + file_path) as file:
                data = json.load(file)
                if data["maintext"] and data["source_domain"]:
                    label = get_label(data["source_domain"])
                    if label is not None:
                        articles.append({"Article_Title": data["title"],
                                         "Article_Text": data["maintext"],
                                         "Publish_Date": data["date_publish"],
                                         "Source": data["source_domain"],
                                         "Language": data["language"],
                                         "Label": label})
                    else:
                        print("Could not get label for source_domain:", data["source_domain"])
                else:
                    print("Missing maintext or source_domain")
        for dir in dirs:
            walk_through_dir(dir)

walk_through_dir(DATA_PATH)
dataset = pd.DataFrame(articles)
print(Counter(dataset.loc[dataset["Language"] == "en"]["Source"]))
print(Counter(dataset.loc[dataset["Language"] == "en"]["Label"]))

In [ ]:
filtered_dataset = pd.DataFrame(columns = ["Article_Title", "Article_Text", "Source", "Label"])

for source in dataset["Source"].unique():
    articles = dataset.loc[(dataset["Source"] == source) & (dataset["Language"] == "en")]
    articles = articles.sample(frac=1)
    articles = articles[:NUM_ARTICLES_PER_DOMAIN]
    filtered_dataset = filtered_dataset.append(articles)
    
print(Counter(filtered_dataset["Source"]))
print(Counter(filtered_dataset["Label"]))

In [ ]:
# Save dataset to disk

print("Saving dataset to disk")
Path(PATH).mkdir(parents=True, exist_ok=True)
dataset.to_csv(PATH + "dataset.csv", index=False)
print("Done")

In [ ]:
for source in filtered_dataset["Source"].unique():
    numWords = filtered_dataset.loc[filtered_dataset["Source"] == source]["Article_Text"].str.split().apply(len)
    plt.bar(source, numWords.mean())
    
plt.title("Average word count per publication")
plt.xticks(filtered_dataset["Source"].unique(), rotation=45)
plt.ylabel("Word count")
plt.xlabel("Publication")
plt.show()

In [ ]:
labelNames = ["Left-Leaning", "Right-Leaning"]
colors = ["red", "blue"]

for label in filtered_dataset["Label"].unique():
    numWords = filtered_dataset.loc[filtered_dataset["Label"] == label]["Article_Text"].str.split().apply(len)
    plt.bar(labelNames[label], numWords.mean(), color=colors[label])
    
plt.title("Average word count per political leaning")
plt.ylabel("Word count")
plt.xlabel("Political Leaning")
plt.show()

In [ ]:
plt.title("Average word count per political leaning")

sourceNames = ["Vox", "NYPost", "HuffPost", "WashTimes", "WaPo", "FoxNews", "NYTimes", "Breitbart"]
labelNames = ["Left-Leaning", "Right-Leaning"]

plt.subplot(2, 2, 1)
for source, sourceName in zip(filtered_dataset["Source"].unique(), sourceNames):
    numWords = filtered_dataset.loc[filtered_dataset["Source"] == source]["Article_Text"].str.split().apply(len)
    plt.bar(sourceName, numWords.mean())
    
plt.ylabel("Word count")
plt.xticks(sourceNames, rotation=75)

plt.subplot(2, 2, 2)
colors = ["red", "blue"]

for label in filtered_dataset["Label"].unique():
    numWords = filtered_dataset.loc[filtered_dataset["Label"] == label]["Article_Text"].str.split().apply(len)
    plt.bar(labelNames[label], numWords.mean(), color=colors[label])

plt.xticks(labelNames, rotation=35)
plt.show()